In [ ]:
from multiprocessing import Queue
from cpu.utils.cpu_monitor_live import run_cpu_monitor, run_pipeline, get_pipeline
from cpu.utils.monitor_simulation import run_cpu_simulation
import pandas as pd

In [ ]:
# create a system model
from cpu.systems import CPUSystem
from cosapp.drivers import NonLinearSolver

cpu = CPUSystem("cpu")
cpu.add_driver(NonLinearSolver("solver", max_iter=10, factor=1.0, tol=1e-6))
cpu["exchanger.h_adder"] = 150
cpu["cpu.heat_capacity"] = 100

In [ ]:
# set process pipeline
queue = Queue()
sequence = {
    "step1": {"nprocs": 8, "duration": 30},
    "step2": {"nprocs": 1, "duration": 30},
}
pipeline, duration = get_pipeline(sequence)

# run pipeline
sim_process = run_cpu_simulation(queue, cpu, duration)
monitor_process = run_cpu_monitor(queue, duration)
run_pipeline(pipeline)
monitor_process.join()
sim_process.join()

# Receive simulation results from the queue
if not queue.empty():
    results = pd.DataFrame(queue.get())

    # Save from notebook (safe path)
    results.to_csv("data/real_time_measure.csv", index=False)
    print("Results saved!")
else:
    print("Queue was empty. No results to save.")

In [ ]:
# Differentiate the simulated and measured data
data_sim = pd.read_csv("data/real_time_measure.csv")
data_mes = pd.read_csv("data/real_time_measure.csv")
data_sim.rename(columns={"T_cpu_simulated": "T_cpu"}, inplace=True)
data_mes.rename(columns={"T_cpu_measured": "T_cpu"}, inplace=True)

In [ ]:
from cpu.utils.plot_recorders import plot_recorders

plot_recorders(
    {
        "operation": data_mes,
        # "simulation": data_sim,
    },
    [[("time", "T_cpu"), ("time", "cpu.usage")], [("T_cpu", "Fan_rpm_1"), ("time", "Fan_rpm_1")]],
    width=600,
    height=350,
)